# Autores colombianos por provincias

In [1]:
import ast
import csv
import os
import re
from collections import defaultdict
from collections import Counter
from datetime import datetime
from datetime import timedelta
from string import punctuation

import matplotlib
import numpy as np
import networkx as nx
import pandas as pd
import requests
import seaborn as sns
from geolocation.main import GoogleMaps
from matplotlib import pyplot as plt
from SPARQLWrapper import SPARQLWrapper, JSON

from googlemaps_key import KEY

## Autores

In [2]:
%%time

autores_df = pd.read_csv(
    'autores7.csv',
    header=0,
    converters={
        'GEOLOC_CAPITAL_AUTOR': ast.literal_eval,
        'GEOLOC_CAPITAL_EJEMPLAR': ast.literal_eval
    }
)
autores_df = autores_df.fillna('')

CPU times: user 3.03 s, sys: 37.9 ms, total: 3.07 s
Wall time: 3.07 s


In [3]:
autores_df.shape

(87200, 9)

In [4]:
autores_df.head()

,TITULO,PAIS_EJEMPLAR,AUTOR,PAIS_AUTOR,FECHA_PUB,CAPITAL_AUTOR,CAPITAL_EJEMPLAR,GEOLOC_CAPITAL_AUTOR,GEOLOC_CAPITAL_EJEMPLAR
0,Adivina en qué pais ...,Spain,Martina Badstuber,Germany,2010,Berlin,Madrid,"(52.52000659999999, 13.404954)","(40.4167754, -3.7037902)"
1,¡No quiero hacer pipí en el orinal!,Spain,Roser Rius,Mexico,2009,Mexico City,Madrid,"(19.4326077, -99.133208)","(40.4167754, -3.7037902)"
2,Humo,Spain,Antón Fortes Torres,Spain,2008,Madrid,Madrid,"(40.4167754, -3.7037902)","(40.4167754, -3.7037902)"
3,Galleta para perros,Spain,Helen Cooper,United Kingdom,2008,London,Madrid,"(51.5073509, -0.1277583)","(40.4167754, -3.7037902)"
4,Autobio,Spain,Cyril Pedrosa,France,2009,Paris,Madrid,"(48.856614, 2.3522219)","(40.4167754, -3.7037902)"


## Autores colombianos

In [5]:
autores_colombianos_df = autores_df[autores_df['PAIS_AUTOR'] == 'Colombia']

In [6]:
autores_colombianos_df.shape

(5551, 9)

In [7]:
autores_colombianos_df.head()

,TITULO,PAIS_EJEMPLAR,AUTOR,PAIS_AUTOR,FECHA_PUB,CAPITAL_AUTOR,CAPITAL_EJEMPLAR,GEOLOC_CAPITAL_AUTOR,GEOLOC_CAPITAL_EJEMPLAR
7,En la laguna más profunda,Colombia,Oscar Collazos,Colombia,2011,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)"
90,El gato bandido y otros cuentos,Colombia,Rafael Pombo,Colombia,2007,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)"
101,Cuentos pintados,Colombia,Rafael Pombo,Colombia,2011,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)"
120,Aleida : 10 años,Colombia,Vladdo,Colombia,2007,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)"
292,Mitos y leyendas indígenas de Colombia,Colombia,Javier Ocampo López,Colombia,2013,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)"


## Departamentos Colombia

In [8]:
deps1_df = pd.read_csv('deps1.csv', sep='\t', header=-1)

In [9]:
deps1_df.shape

(33, 8)

In [10]:
deps1_df.head()

,0,1,2,3,4,5,6,7
0,0,Capital District,Enrique Peñalosa,CR,Bogotá,"1,587","8,254,722",1538
1,1,Amazonas,Manuel Antonio Carebilla Cuéllar,CR,Leticia,"109,665","80,360",1991
2,2,Antioquia,Luis Pérez,Liberal,Medellín,"63,612","5,750,478",1826
3,3,Arauca,Ricardo Alvarado Bestene,La U,Arauca,"23,818","282,302",1991
4,4,Atlántico,Eduardo I. Verano de la Rosa,Liberal,Barranquilla,"3,388","2,365,663",1910


In [11]:
deps1_df[1] = deps1_df.apply(lambda x: x[1].strip(), axis=1)

In [12]:
deps1_set = set(deps1_df[1])

In [13]:
deps2_df = pd.read_csv('deps2.txt', header=-1)

In [14]:
deps2_df.shape

(33, 1)

In [15]:
deps2_df.head()

,0
0,Antioquia
1,Caldas
2,Bogotá D.C.
3,Bolívar
4,Córdoba


In [16]:
deps2_df[0] = deps2_df.apply(lambda x: x[0].strip(), axis=1)

In [17]:
deps2_set = set(deps2_df[0])

In [18]:
deps1_set.union(deps2_set).difference(deps1_set.intersection(deps2_set))

{'Bogotá D.C.', 'Capital District'}

In [19]:
deps_mappings = {}
for d in deps1_set:
    deps_mappings[d] = d
deps_mappings['Capital District'] = 'Bogotá D.C.'

## Ciudades y departamentos

In [20]:
dtypes = {
    'Ciudad': str,
    'Población': np.int32,
    'Departamento': str,
}

ciudades_df = pd.read_csv(
    'ciudades_colombia.csv',
    sep='\t',
    header=-1,
    names=['Wiki index', 'Ciudad', 'Población', 'Departamento'],
    usecols=['Ciudad', 'Población', 'Departamento'],
    dtype=dtypes,
    thousands='.',
)

In [21]:
departamentos_sr = pd.Series(list(ciudades_df['Departamento']), index=ciudades_df['Ciudad'])

## Ciudades y departamentos autores

In [22]:
def get_ciudad_departamento_autor_data_wkd(name, lang):
    template = """
        PREFIX wd: <http://www.wikidata.org/entity/>
        PREFIX wdt: <http://www.wikidata.org/prop/direct/>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

        SELECT ?authorLabel ?pobLabel ?depLabel
        WHERE {{
            BIND("{0}"@{1} AS ?authorLabel) .
            ?author rdfs:label ?authorLabel .
            ?author wdt:P31 wd:Q5 .
            OPTIONAL {{ ?author wdt:P106 wd:Q36180 . }}
            ?author wdt:P19 ?pob .
            ?pob wdt:P31 wd:Q2555896 .
            ?pob rdfs:label ?pobLabel .
            ?pob wdt:P131 ?dep .
            ?dep wdt:P31 wd:Q215655 .
            ?dep rdfs:label ?depLabel .
            FILTER(lang(?pobLabel)="{1}")
            FILTER(lang(?depLabel)="{1}")
        }}
    """
    query = template.format(name, lang)
    url = 'https://query.wikidata.org/bigdata/namespace/wdq/sparql'
    data = requests.get(url, params={'query': query, 'format': 'json'}).json()
    return data

In [32]:
sparql_dbp = SPARQLWrapper("http://dbpedia.org/sparql")

def get_ciudad_departamento_autor_data_dbp(name, lang):
    template = """
        PREFIX dbo: <http://dbpedia.org/ontology/>
        PREFIX dbp: <http://dbpedia.org/property/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX yago: <http://dbpedia.org/class/yago/>

        SELECT ?authorLabel ?pobLabel ?depLabel
        WHERE {{
            BIND("{0}"@{1} AS ?authorLabel) .
            ?author rdfs:label ?authorLabel .
            ?author rdf:type dbo:Person .
            OPTIONAL {{ ?author rdf:type dbo:Writer . }}
            ?author dbp:birthPlace ?pob .
            ?pob rdf:type dbo:Place .
            ?pob rdfs:label ?pobLabel .
            ?pob dbo:isPartOf ?dep .
            ?dep rdf:type yago:WikicatDepartmentsOfColombia .
            ?dep rdfs:label ?depLabel .
            FILTER(lang(?pobLabel)="{1}")
            FILTER(lang(?depLabel)="{1}")
        }}
    """
    query = template.format(name, lang)
    sparql_dbp.setQuery(query)
    sparql_dbp.setReturnFormat(JSON)
    data = sparql_dbp.query().convert()
    return data

In [24]:
def get_ciudad_departamento_autor_data(db, name, lang):
    try:
        if db == 'wdt':
            data = get_ciudad_departamento_autor_data_wkd(name, lang)
        elif db == 'dbp':
            data = get_ciudad_departamento_autor_data_dbp(name, lang)

        pobs = set(d['pobLabel']['value'] for d in data['results']['bindings'])
        if len(pobs) == 0:
            pob = 'UNKOWN'
        elif len(pobs) == 1:
            pob = pobs.pop()
        else:
            print('MáS DE UNA POBLACIóN PARA', name, ':', pobs)
            pob = pobs.pop()

        deps = set(d['depLabel']['value'] for d in data['results']['bindings'])
        if len(deps) == 0:
            dep = 'UNKNOWN'
        elif len(deps) == 1:
            dep = deps.pop()
        else:
            print('MáS DE UN DEPARTAMENTO PARA', name, ':', deps)
            dep = deps.pop()
    except:
        print('ERROR EN SPARQL PARA', name)
        pob = 'UNKNOWN'
        dep = 'UNKNOWN'
    return (pob, dep)

In [25]:
def get_ciudad_departamento_autor(name):
    for lang in ['es', 'en']:
        for db in ['wkd', 'dbp']:
            r = get_ciudad_departamento_autor_data(db, name, lang)
            if r != ('UNKNOWN', 'UNKNOWN'):
                return r
    return ('DESCONOCIDO', 'DESCONOCIDO')

In [ ]:
%%time

autores_ciudades_departamentos_dict = {}

for x in autores_colombianos_df['AUTOR'].unique():
    autores_ciudades_departamentos_dict[x] = get_ciudad_departamento_autor(x)

ERROR EN SPARQL PARA Oscar Collazos
ERROR EN SPARQL PARA Rafael Pombo
ERROR EN SPARQL PARA Vladdo
ERROR EN SPARQL PARA Javier Ocampo López
ERROR EN SPARQL PARA Germán Arciniegas
ERROR EN SPARQL PARA Luis Carlos Restrepo
ERROR EN SPARQL PARA Carmelina Soto Valencia
ERROR EN SPARQL PARA Jesús Arango Cano
ERROR EN SPARQL PARA Luis Duque Gómez
ERROR EN SPARQL PARA Manuel Mejía Vallejo
ERROR EN SPARQL PARA José Eustasio Rivera
ERROR EN SPARQL PARA Gabriel García Márquez
ERROR EN SPARQL PARA Gonzalo Correal Urrego
ERROR EN SPARQL PARA Celso Román
ERROR EN SPARQL PARA Juan Gustavo Cobo Borda
ERROR EN SPARQL PARA Claudia Rueda
ERROR EN SPARQL PARA Giovanni Quessep
ERROR EN SPARQL PARA Mauricio Vélez
ERROR EN SPARQL PARA Juan Manuel Roca
ERROR EN SPARQL PARA José María Vargas Vila
ERROR EN SPARQL PARA Pedro Ortiz Valdivieso
ERROR EN SPARQL PARA Andrés Caicedo Estela
ERROR EN SPARQL PARA Gustavo Adolfo Garcés
ERROR EN SPARQL PARA Ricardo Silva Romero
ERROR EN SPARQL PARA Germán Castro Caycedo
ER

In [ ]:
%%time

def process_ciudad(x):
    return autores_ciudades_departamentos_dict[x['AUTOR']][0]

autores_colombianos_df['CIUDAD_AUTOR'] = autores_colombianos_df.apply(process_ciudad, axis=1)

In [ ]:
%%time

def process_departamento(x):
    return autores_ciudades_departamentos_dict[x['AUTOR']][1]

autores_colombianos_df['CIUDAD_AUTOR'] = autores_colombianos_df.apply(process_departamento, axis=1)

# Saving results

In [41]:
autores_colombianos_df.to_csv('autores_colombianos_1.csv', index=False)